In [1]:
import json
import os
import pandas as pd

In [2]:
os.chdir('../')

In [3]:
from src.data.ClickhouseClient import ClickhouseClient

In [4]:
# Get Clickhouse credentials and connect
filename_credentials = 'clickhouse_credentials.json'
with open(filename_credentials, 'r') as f:
    credentials = json.load(f)

In [7]:
client = ClickhouseClient(
    host = credentials['host'],
    port = credentials['port'],
    user = credentials['user'],
    password = credentials['password'],
    database = credentials['database']
)

In [8]:
client.create_database('test')

In [27]:
client.query_dataframe('SELECT * FROM raw.meal LIMIT 10')

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian
5,1311,Extras,Thai
6,1062,Beverages,Italian
7,1778,Beverages,Italian
8,1803,Extras,Thai
9,1198,Extras,Thai


In [22]:
client.drop_table(database='test', table_name='test_join')

In [23]:
client.execute('CREATE TABLE IF NOT EXISTS test.test_join ENGINE = Memory AS (SELECT * FROM raw.meal)')

[]

In [24]:
client.execute('SHOW TABLES IN test')

[array(['test_join'], dtype=object)]

In [26]:
client.query_dataframe('SELECT * FROM test.test_join')

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian
5,1311,Extras,Thai
6,1062,Beverages,Italian
7,1778,Beverages,Italian
8,1803,Extras,Thai
9,1198,Extras,Thai


In [33]:
client.query_dataframe("""
	SELECT
		t.id,
		t.week,
		t.center_id,
		t.meal_id,
		t.checkout_price,
		t.base_price,
		t.emailer_for_promotion,
		t.homepage_featured,
		t.num_orders,
		c.city_code,
		c.region_code,
		c.center_type,
		c.op_area,
		m.category,
		m.cuisine
	FROM raw.train t
	LEFT JOIN raw.center c ON c.center_id = t.center_id
	LEFT JOIN raw.meal m ON m.meal_id = t.meal_id
""")

,t_id,t_week,t_center_id,t_meal_id,t_checkout_price,t_base_price,t_emailer_for_promotion,t_homepage_featured,t_num_orders,c_city_code,c_region_code,c_center_type,c_op_area,m_category,m_cuisine
0,1379560,1,55,1885,136.830002,152.289993,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.830002,135.830002,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.860001,135.860001,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.500000,437.529999,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.500000,242.500000,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,1271326,145,61,1543,484.089996,484.089996,0,0,68,473,77,TYPE_A,4.5,Desert,Indian
456544,1062036,145,61,2304,482.089996,482.089996,0,0,42,473,77,TYPE_A,4.5,Desert,Indian
456545,1110849,145,61,2664,237.679993,321.070007,0,0,501,473,77,TYPE_A,4.5,Salad,Italian
456546,1147725,145,61,2569,243.500000,313.339996,0,0,729,473,77,TYPE_A,4.5,Salad,Italian
